## 2. Working with files

<img src="https://tuplex.cs.brown.edu/_static/img/logo.png" width="128px" style="float: right;" />

In the 2nd part of the Tuplex intro series, we'll take a look at how to work with CSV and text files.

### 2.1 Basic IO - Reading CSV files
To read in a csv file, Tuplex provides an API function `csv`

In [ ]:
import tuplex

c = tuplex.Context({'tuplex.redirectToPythonLogging':False})

In the following cells, we use some sample data from Google Colab. We can simply load it into Tuplex using the `csv` command.

In [ ]:
ds = c.csv('sample_data/california_housing_train.csv')

In [ ]:
ds.show(5)

Without any further information, Tuplex automatically deduces types for each column. In order to check what types Tuplex deduced, we can use the `columns` and `types` properties of a Tuplex dataset.

In [ ]:
columns = ds.columns
types = ds.types

# print out as nicely formatted dictionary
dict(zip(columns, types))

Sometimes however, it may be desirable to assign specific types to individual columns. Luckily, Tuplex provides a mechanism for this as well:

In [ ]:
c.csv('sample_data/california_housing_train.csv',  type_hints={'longitude' : float, 'latitude' : str}).show(4)

Let's say we now want to create a file containing only data entries where the `housing_median_age` is larger than `50`:

In [ ]:
ds.filter(lambda r: r['housing_median_age'] > 50).tocsv('lt50.csv', num_parts=0)

In order to speedup data output, Tuplex by default uses multiple threads to create multiple output parts.

In [ ]:
!head lt50.part0.csv

Besides CSV files, Tuplex also has experimental support to read/write [ORC files](https://https://orc.apache.org/), which may be a more space efficient solution depending on the data and workload.

In [ ]:
ds.toorc('lt50.orc')

Similarly, the orc files can be read using the `orc` command.

In [ ]:
c.orc('lt50.part0.orc').show(5)

## 2.2 Working with larger files
Naturally, the benefit of Tuplex's compilation comes into play when working with larger files. To demonstrate this, let's assume we want to work with the 311 original data. A subset of this (1GB, ~212MB to download) can be downloaded via the following command

In [ ]:
!gdown https://drive.google.com/uc?id=18e2GyoQKLnQ2_uaUcaSOsLRlIT-7tqpN && tar xf 311_subset.tar.gz && mv 311_subset.csv sample_data/

Next, let's create a new context with more memory to process the larger file. You can still reuse the old one albeit at the cost of incurring a lot of disk swapping. Therefore, we delete the old context to free up the space.

In [ ]:
del c

In [ ]:
!head sample_data/311_subset.csv

In [ ]:
c = tuplex.Context({'tuplex.redirectToPythonLogging':True, 'tuplex.executorMemory':'3G', 'tuplex.driverMemory':'3G'})

Again, we can use Tuplex's autodetection feature to load the file and assign meaningful default types.

In [ ]:
ds = c.csv('sample_data/311_subset.csv')

In [ ]:
dict(zip(ds.columns, ds.types))

Executing a simple query on the input data creates a logical plan under the hood, which then gets optimized into a physical plan together with auto-generated efficient code that gets lowered ultimately to native code optimized for the machine it is executed on.

In [ ]:
ds.selectColumns(['Unique Key']).show(5)

As for every operation, we can retrieve help using Python's builtin documentation featue.

In [ ]:
help(ds.selectColumns)

I.e., when looking up the semantics of the `selectColumns` operation, it's also possible to use integers instead of strings to select columns for more flexibility.

In [ ]:
ds.selectColumns([0, 1]).show(3)

Let's say, we want to use a slightly more complicated pipeline now. As an initial step, let's first investigate what kind ofcomplaint types there are. To find the corresponding column, we can use the meta-data associated with a dataset and then design a first, exploratory query.

In [ ]:
def print_table(arr, break_after=5):
    for i in range(len(arr) // break_after +1):
        print(' | '.join(arr[i * break_after:(i +1)* break_after]))

print_table(ds.columns)

In [ ]:
complaint_types = ds.selectColumns(['Complaint Type']).unique().collect()

In [ ]:
print(complaint_types)

Looking at the data, we see that there are some complaints regarding mosquitoes. Likely, because it gets quite hot and humid in summer in New York City! Can the data back this up?

To find out, let's plot the number of mosquito complaints per month for the last year. A helpful function for aggregating the results is `aggregateByKey`:

In [ ]:
help(ds.aggregateByKey)

Next, let's use a UDF to extract the month and year of the complaint and limit the search to complain types so Tuplex automatically processes fewer rows.

In [ ]:
ds.selectColumns(['Created Date']).show(5)

In [ ]:
year_to_investigate = 2019

def extract_month(row):
    date = row['Created Date']
    date = date[:date.find(' ')]
    return int(date.split('/')[0])

def extract_year(row):
    date = row['Created Date']
    date = date[:date.find(' ')]
    return int(date.split('/')[-1])

ds2 = ds.withColumn('Month', extract_month) \
  .withColumn('Year', extract_year) \
  .filter(lambda row: 'Mosquito' in row['Complaint Type']) \
  .filter(lambda row: row['Year'] == year_to_investigate) \
  .selectColumns(['Month', 'Year', 'Complaint Type'])


ds2.show(5)


We can now use the aggregateByKey function to count the number of mosquito complaints per month in 2019.

In [ ]:
def combine_udf(a, b):
    return a + b

def aggregate_udf(agg, row):
    return agg + 1

ds2.aggregateByKey(combine_udf, aggregate_udf, 0, ["Month"]).show()

Yet, it seems that mosquito complaints are actually not that common. In total there are 4 complaints for the whole year, of which 3 are in December. Thus we actually can't draw with such little support any meaningful conclusions about mosquitos in NYC from the 311 dataset.

Let's step back and check actually, what kind of complaint is actually the most common:

In [ ]:
data = ds.aggregateByKey(combine_udf, aggregate_udf, 0, ["Complaint Type"]).collect()

To see what the most common complaint is, let's sort the output:

In [ ]:
data = sorted(data, key=lambda x: -x[1])

data[:5]

As we can see,  is the most common complaint and with a little more code, a plot can be generated - can you do it?

(c) 2017 - 2022 Tuplex team